# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [31]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('mylogreg consulting project').getOrCreate()
from pyspark.ml.classification import LogisticRegression
data = spark.read.csv('customer_churn.csv', inferSchema=True, header=True)
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [32]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [33]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [35]:
from pyspark.ml.feature import VectorAssembler
#company_indexer = StringIndexer(inputCol='Company', outputCol='Company_index')
#company_encoder = OneHotEncoder(inputCol='Company_index', outputCol='Company_vector')
assembler = VectorAssembler(inputCols=['Age',
                                       'Total_Purchase',
                                       'Account_Manager',
                                       'Years',
                                       'Num_Sites'],
                            outputCol='features')
assembled = assembler.transform(data)
final_data = assembled.select('features', 'Churn')
from pyspark.ml.classification import LogisticRegression
#from pyspark.ml import Pipeline
log_reg_customers = LogisticRegression(labelCol='Churn')
#pipeline = Pipeline(stages=[company_indexer, company_encoder, assembler, log_reg_customers])
#pipeline = Pipeline(stages=[assembler, log_reg_customers])
train_data, test_data = final_data.randomSplit([0.7, 0.3])
train_data.describe().show()
test_data.describe().show()

+-------+------------------+
|summary|             Churn|
+-------+------------------+
|  count|               617|
|   mean|0.1507293354943274|
| stddev| 0.358075147646493|
|    min|                 0|
|    max|                 1|
+-------+------------------+

+-------+-------------------+
|summary|              Churn|
+-------+-------------------+
|  count|                283|
|   mean|0.20141342756183744|
| stddev|0.40176664228694037|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



In [36]:
fit_model = log_reg_customers.fit(train_data)
results = fit_model.transform(test_data)
results.select('Churn','prediction').show()

+-----+----------+
|Churn|prediction|
+-----+----------+
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
+-----+----------+
only showing top 20 rows



In [38]:
training_sum = fit_model.summary
training_sum.predictions.describe().show()

+-------+------------------+-------------------+
|summary|             Churn|         prediction|
+-------+------------------+-------------------+
|  count|               617|                617|
|   mean|0.1507293354943274| 0.1085899513776337|
| stddev| 0.358075147646493|0.31137648230081566|
|    min|               0.0|                0.0|
|    max|               1.0|                1.0|
+-------+------------------+-------------------+



In [45]:
pred_and_labels = fit_model.evaluate(test_data)
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|Churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,1....|    0|[5.41976804204140...|[0.99559134859526...|       0.0|
|[28.0,8670.98,0.0...|    0|[8.88106963743531...|[0.99986102390571...|       0.0|
|[28.0,11245.38,0....|    0|[4.29283641490091...|[0.98651813706769...|       0.0|
|[29.0,5900.78,1.0...|    0|[4.29718530744897...|[0.98657585557084...|       0.0|
|[29.0,8688.17,1.0...|    1|[2.95864038249674...|[0.95067027193227...|       0.0|
|[29.0,9378.24,0.0...|    0|[5.55846058001770...|[0.99616009585725...|       0.0|
|[29.0,12711.15,0....|    0|[6.23559962497265...|[0.99804537352313...|       0.0|
|[29.0,13255.05,1....|    0|[4.89127323549914...|[0.99254415517614...|       0.0|
|[30.0,7960.64,1.0...|    1|[3.88736725324108...|[0.97991253350883...|       0.0|
|[30.0,8677.28,1

In [47]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
meval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Churn')
AUC = meval.evaluate(pred_and_labels.predictions)
AUC

0.8089970501474926

In [49]:
final_lr_model = log_reg_customers.fit(final_data)
new_customers = spark.read.csv('new_customers.csv', inferSchema=True, header=True)
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [51]:
test_new_customers = assembler.transform(new_customers)
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [53]:
final_results = final_lr_model.transform(test_new_customers)
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



In [54]:
test_new_customers.describe().show()

+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
|summary|        Names|               Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|         Company|
+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
|  count|            6|                 6|                6|                 6|                6|                 6|                   6|               6|
|   mean|         null|35.166666666666664|7607.156666666667|0.8333333333333334|6.808333333333334|12.333333333333334|                null|            null|
| stddev|         null| 15.71517313511584|4346.008232825459| 0.408248290463863|3.708737880555414|3.3862466931200785|                null|            null|
|    min|Andrew Mccall|              22.0|            100.0|          